In [ ]:
import requests
from pymongo import MongoClient
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import urllib2
import sys

## To write down all the urls to be scraped for articles and comments

In [ ]:
# url = "https://www.theguardian.com/us/commentisfree"

# Which links to look for.

# years = ["2015", "2016", "2017"]
# months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
# dates = [str(n) for n in xrange(1,28)]

years = ["2017"]
months = ["oct"]
dates = [str(31)]


# count of number of links
i = 0

# Initiating mongoDB. Using pymongo to connect the database
client = MongoClient()
db = client["guardian"] #This is the name of the database
urls = db["urls"] # this is the table in that database

result = db.urls.delete_many({}) # A fresh start to the DB table -> removing all entries 
all_links = []

for month in months:
    for date in dates:
        root_url = "https://www.theguardian.com/commentisfree/2016/" + month + "/" + date 
        
        # This one contains all the URLs. Soup extracts those.
        list_of_urls = requests.get(root_url).text
        soup = BeautifulSoup(list_of_urls, "html.parser")

        # the .findAll method from re finds the _content_ that has 
        # html tag - 'a', attribute - 'href' and 
        # the pattern that begins with the root_url variable
        for link in soup.findAll('a', attrs={'href': re.compile(root_url)}):
            all_links.append(link.get('href'))
            
all_links = set(all_links)
for link in all_links:
    # urls is the name of the mongo DataBase. insert_one is the method 
    urls.insert_one({"id" : i, "url" : link})
#             print link
    i += 1
print "Total number of links stored is ", i

## To download comments from guardian website

In [ ]:
def download_comments(guardianLink):
    '''
    This function downloads the comments, along with title, topic of the original thread
    comment_id, no_upvotes, no_replies, comment_author, comment_author_id, etc
    Returns BeautifulSoup; need more extraction from the soup for the above-mentioned 
    parameters
    
    '''
        
    #Figuring out the PageId
    pageResponse = urllib2.urlopen(guardianLink)
    commentsMatch = re.search(r'/p/(.*?)"', pageResponse.read(), re.M|re.I)
    # print "Comments Match = ", commentsMatch()
    if commentsMatch:
        pageId = commentsMatch.group(1)
        print '[+] pageId has been retrieved ('+pageId+')'
    else:
        sys.exit('[-] Could not retrieve pageId!')

    #Retrieving comments
    downloadCount = 1
    downloadError = 0

    text = ''

    while downloadError == 0:
        try:
            response = urllib2.urlopen('http://www.theguardian.com/discussion/p/' + pageId 
                                       + '?page=' + str(downloadCount))
            html = response.read()
            length = len(text)
            text += html
            downloadCount = downloadCount + 1

        except:
            downloadError = 1

    return BeautifulSoup(text, 'html.parser')

## To extract data from soup

In [ ]:
startTime = time.time()
stopTime = time.time()
runningTime = (stopTime - startTime)/60;
print '[=] ' + str(downloadCount-1) + ' pages have been downloaded in ' + str(round(runningTime,2)) + ' minutes'
